In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [15]:
new_df = df.sample(10000)

In [16]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [17]:
new_df.duplicated().sum()

0

In [18]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
342086,What is the difference between host and server...,What is the difference between hosting and a s...
281674,"If human beings had wings, how large would the...",What would a large scale explosion be like on ...
297285,How can I increase the traffic to my website?,How can I Increase the traffic of my blog?
86878,Where is the best place to live in San Francisco?,Where is the best place to stay in San Francis...
155250,Which country is best for an M.arch? Do B.arch...,Is it worth doing m.arch from USA?


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=1000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [20]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(10000, 2000)

In [32]:
cv.get_feature_names_out()[878]

'then'

In [34]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,is_duplicate
342086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
86878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
155250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
134224,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
312598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291554,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [23]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,is_duplicate
342086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
86878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
155250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7285

In [29]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7265